In [19]:
import pandas as pd, datetime
import numpy as np
import glob
import os
import seaborn as sns
import sys, getopt

In [6]:
emp = pd.read_csv(r'D:\data\salesperson\salesperson.csv')

In [142]:
emp.head()

,Salesman,Salesman No,Is Void,POS,Barcode,Store,Division,Section,Department,Sale Qty,Net Amt,Rate,Bill Date
0,NITESH KUMAR SINGH,36.0,No,CMHOSTR03-999288,C802999,Hatibagan Stockpoint,Accoessories,Gift & Novelties,Baby Accessory,1.0,55.0,34.96,01-11-2023
1,LALIT AGARWAL (PS),73.0,No,CMHOSTR04-462155,C793771,Garihat Stock Point,Accoessories,Gift & Novelties,Baby Accessory,1.0,99.0,48.66,01-11-2023
2,PUJA PASHI,34.0,No,CMHOSTR03-999258,C802984,Hatibagan Stockpoint,Accoessories,Gift & Novelties,Baby Accessory,2.0,398.0,103.06,01-11-2023
3,BAPI TARAFDER (PS),14.0,No,CMHOSTR04-462175,C766996,Garihat Stock Point,Accoessories,Gift & Novelties,Books,1.0,150.0,112.50,01-11-2023
4,FMCG BRAND,111.0,No,CMHOSTR01-2256825,C800062,Newmarket Stock Point,Accoessories,Gift & Novelties,Dry Flower,1.0,139.0,65.00,01-11-2023


In [143]:
grouped = emp.groupby(emp.Store)

In [144]:
emp.Store.unique()

array(['Hatibagan Stockpoint', 'Garihat Stock Point',
       'Newmarket Stock Point', 'Megashop Stock', nan], dtype=object)

In [145]:
emp_hatibagan = grouped.get_group("Hatibagan Stockpoint")
emp_lighthouse = grouped.get_group("Newmarket Stock Point")
emp_chowrunghee = grouped.get_group("Megashop Stock")

In [146]:
l = emp_hatibagan.Department.nunique()
print(l)

478


In [147]:
df = emp_hatibagan.groupby('Salesman').agg(Lines=('Department', 'nunique'), Total_Qty=('Sale Qty', 'sum'), 
                                            Sale_Amt=('Net Amt', 'sum'))

In [148]:
df.head()

,Lines,Total_Qty,Sale_Amt
Salesman,,,
ANUSHMITA PAUL,6,11.0,3096.00
ARDHENDU kr SARDAR,3,3.0,925.00
ARINDAM BOSE,111,1225.0,457399.06
BAISHALI DAS,237,2625.0,961271.20
Blank / Brand,1,1.0,999.00


In [149]:
s = df.Sale_Amt.sum()
s

14830190.04

In [150]:
q = df.Total_Qty.sum()
print(q)

60211.0


In [151]:
df['percent_of_Sales_amt']= df['Sale_Amt']/s *100
df['percentage_Qty'] = df['Total_Qty']/q *100
df['percentage_lines'] = df['Lines']/l *100

In [152]:
df.head()

,Lines,Total_Qty,Sale_Amt,percent_of_Sales_amt,percentage_Qty,percentage_lines
Salesman,,,,,,
ANUSHMITA PAUL,6,11.0,3096.00,0.020876,0.018269,1.255230
ARDHENDU kr SARDAR,3,3.0,925.00,0.006237,0.004982,0.627615
ARINDAM BOSE,111,1225.0,457399.06,3.084243,2.034512,23.221757
BAISHALI DAS,237,2625.0,961271.20,6.481854,4.359668,49.581590
Blank / Brand,1,1.0,999.00,0.006736,0.001661,0.209205


In [153]:
df.percentage_Qty.sum()

100.00000000000001

In [154]:
df['Score'] = (0.50 * df['percent_of_Sales_amt']) + (0.25 * df['percentage_Qty']) + (0.25 * df['percentage_lines'])

In [155]:
df.head()

,Lines,Total_Qty,Sale_Amt,percent_of_Sales_amt,percentage_Qty,percentage_lines,Score
Salesman,,,,,,,
ANUSHMITA PAUL,6,11.0,3096.00,0.020876,0.018269,1.255230,0.328813
ARDHENDU kr SARDAR,3,3.0,925.00,0.006237,0.004982,0.627615,0.161268
ARINDAM BOSE,111,1225.0,457399.06,3.084243,2.034512,23.221757,7.856189
BAISHALI DAS,237,2625.0,961271.20,6.481854,4.359668,49.581590,16.726241
Blank / Brand,1,1.0,999.00,0.006736,0.001661,0.209205,0.056085


In [156]:
df.Score.sum()

290.84728033472805

In [157]:
df.percentage_lines.sum()

863.389121338912

In [158]:
df.percentage_Qty.sum()

100.00000000000001

In [159]:
df.percent_of_Sales_amt.sum()

100.0

In [160]:
df = df.sort_values('Score' , ascending = False)

In [161]:
df.head()

,Lines,Total_Qty,Sale_Amt,percent_of_Sales_amt,percentage_Qty,percentage_lines,Score
Salesman,,,,,,,
PUJA PASHI,328,9128.0,1192105.63,8.038371,15.160021,68.619247,24.964002
subrata das,255,10720.0,833278.70,5.618800,17.804056,53.347280,20.597234
PAMPA PAUL SHOW,295,3726.0,951332.05,6.414834,6.188238,61.715481,20.183347
SUMIT BISWAS,269,3241.0,730652.71,4.926793,5.382737,56.276151,17.878118
SANCHALI MAITY,258,2395.0,854948.65,5.764920,3.977678,53.974895,17.370604


In [162]:
df.to_excel('Hatibagan_employee.xlsx')